In [1]:
import matplotlib.pyplot as plt
import numpy as np

def precipitations(year,month,day):                    # Cette fonction permet simplement de représenter l'évolutiond es précipitations en Ile-de-France sur une journée.
    path = 'data/'+'{:04d}/'.format(year)
    file = 'RR_IDF300x300_{:04d}{:02d}{:02d}.npy'.format(year,month,day)
    file = path + file
    RR = np.load(file)/100.0
    RR[RR < 0]=np.nan
    plt.figure()
    plt.ion()        
    for i in range (288):
        plt.imshow(RR[i,:,:], cmap='Blues')
        #plt.show(block=True)
        #cv2.namedWindow('image', cv2.WINDOW_NORMAL)
        #cv2.imshow('image', RR[i,:,:], cmap='Blues')
        plt.pause(0.2)
        plt.clf()

def segmentation_evenements(year,month,day):        # Cette fonction permet de séparer les différents événements de précipitations.
    path = 'data/'+'{:04d}/'.format(year)
    file = 'RR_IDF300x300_{:04d}{:02d}{:02d}.npy'.format(year,month,day)
    file = path + file
    RR = np.load(file)/100.0
    RR[RR < 0]=np.nan
    RR_seg = np.zeros([288,300,300])        # RR_seg sera une carte simplifiée indiquant simplement où un événement de précpitation est en cours. C'est un tableau binaire.
    for i in range (300):
        for j in range (300):
            t = 0
            t1 = 0
            while (t<288):
                if (RR[t,i,j]>0):
                    RR_seg[t,i,j] = 1
                    t1 = 0      # Compteur du temps à partir du dernier instant de pluie. S'il atteint 6, alors l'événement de pluie est terminé.
                    t = t+1
                    t0 = t      # t0 permet de garder en mémoire le premier instant de l'acalmie.
                    while (t<288 and (RR[t,i,j]==np.nan) and t1<6):      # Deux averses au même endroit font partie du même événement pluvieux si elles adviennent à moins de 30 min d'intervalle.
                        t1 = t1+1
                        t = t+1
                    if (t<288 and t1<6):
                        for t2 in range (t0,t+1):
                            RR_seg[t2,i,j] = 1
                else :
                    t = t+1
    return RR_seg

In [15]:
# each (i, j) in a matrix is either 1 if an event is in progress, or 0 if there is no event. We want to compute the length of events over the whole year.
# the result is a matrix of the same size as the input, where each (i, j) is a list of tuples (time of event, length of event). Each tuple represent an event at (i,j)
def event_length(year, month, day):
    RR_seg = segmentation_evenements(year, month, day)
    length = np.zeros([300,300], dtype=object)
    for i in range(300):
        for j in range(300):
            t = 0
            events = []
            while t < 288:
                if RR_seg[t, i, j] == 1:
                    t0 = t
                    while t < 288 and RR_seg[t, i, j] == 1:
                        t += 1
                    events.append((t0, t - t0))
                t += 1
            length[i, j] = events
    return length


i, j = 100, 150
events = event_length(2018, 1, 1)[i, j]


In [21]:
# for a given event, we want to compute the maximum intensity, the mean intensity, its variance, and the percentage of null values
def event_stats(year, month, day, i, j, start_time, duration):
    path = 'data/'+'{:04d}/'.format(year)
    file = 'RR_IDF300x300_{:04d}{:02d}{:02d}.npy'.format(year,month,day)
    file = path + file
    RR = np.load(file)/100.0
    RR[RR < 0]=np.nan
    RR_event = RR[start_time:start_time+duration, i, j]
    RR_event = RR_event[~np.isnan(RR_event)]
    print(RR_event)
    return (duration, np.max(RR_event), np.mean(RR_event), np.var(RR_event), 1 - np.count_nonzero(RR_event) / duration)

In [22]:
for e in events:
    start_time = e[0]
    duration = e[1]
    print(event_stats(2018, 1, 1, i, j, start_time, duration))

[ 0.36  0.84  0.6   1.79  1.68  3.36  3.36  4.68  2.88  2.4   6.48 10.32
  4.8   8.76  5.16  4.92  8.04  9.72  2.28  3.59  2.76  4.08  6.84  4.8
  2.4   0.96  0.24  1.32  1.08  0.  ]
(30, 10.32, 3.6833333333333327, 7.959862222222223, 0.033333333333333326)
[0.24 0.72 0.72 1.08 1.08 0.  ]
(6, 1.08, 0.64, 0.1616, 0.16666666666666663)
[0.36 2.16 1.56 0.36 0.36 0.36 0.24 0.  ]
(8, 2.16, 0.6749999999999999, 0.5037750000000001, 0.125)
[1.92 0.  ]
(2, 1.92, 0.96, 0.9216, 0.5)
[0.24 0.96 0.84 1.44 1.08 2.04 0.96 0.  ]
(8, 2.04, 0.945, 0.35797500000000004, 0.125)
[0.24 0.24 0.  ]
(3, 0.24, 0.16, 0.012799999999999999, 0.33333333333333337)
[0.24 0.  ]
(2, 0.24, 0.12, 0.0144, 0.5)
